# Get Wiki

In [19]:
from WikiComponents import Article
from helper import read_csv

article_list = read_csv("data/articles.csv",
                        lambda row: Article(id_=row[1], grade=row[2]))

# Group those `good` and `not-so-good` articles
groupGood = list(filter(lambda i: i.grade in ['A', 'FA', 'GA'], article_list))
groupNSGood = list(filter(lambda i: i.grade in ['C', 'Start'], article_list))

print('groupGood: n =', len(groupGood))
print('groupNSGood: n =', len(groupNSGood))

groupGood: n = 152
groupNSGood: n = 250


In [20]:
article = groupGood[0]

## Unique Contributors

In [ ]:
from WikiComponents import WikiAPI
from helper import get_first_query_page

# todo Use combined query makes more efficient.
_url = "api.php?action=query&titles={article_id}&prop=contributors"

resp = WikiAPI.get(_url.format(article_id=article.id_))
first = get_first_query_page(resp)
n_contributers_a = first['anoncontributors']
n_contributers = len(first['contributors'])
{'anoncontributors': n_contributers_a, 'contributors': n_contributers}

[i] requesting "https://en.wikipedia.org/w/api.php?action=query&titles=Parallel computing&prop=contributors" with param {'format': 'json'}


{'anoncontributors': 298, 'contributors': 10}

## Editor Revisions

In [22]:
_url = 'api.php?action=query&prop=revisions&titles={article_id}' \
       '&rvprop=timestamp|user|userid&rvlimit=max'
dict_rev_info = {}


def proc(resp: dict):
    """
    "revisions": [
    {
        "user": "74.96.187.144",
        "anon": "",
        "userid": 0,
        "timestamp": "2009-11-22T17:04:09Z"
    },
    {
        "user": "Miym",
        "userid": 8436643,
        "timestamp": "2009-07-30T15:35:49Z"
    }
    """
    first = get_first_query_page(resp)
    for rv in first['revisions']:
        uid = rv['userid']
        if uid in dict_rev_info:
            dict_rev_info[uid].append(rv)
        else:
            dict_rev_info[uid] = [rv]
            
            
u = _url.format(article_id=article.id_)
resp = WikiAPI.get(u)
proc(resp)

# Pagination
while resp.get('continue') and resp['continue'].get('rvcontinue'):
    resp = WikiAPI.get(u, {'rvcontinue': resp['continue']['rvcontinue']})
    proc(resp)

# print(dict_rev_info)

[i] requesting "https://en.wikipedia.org/w/api.php?action=query&prop=revisions&titles=Parallel computing&rvprop=timestamp|user|userid&rvlimit=max" with param {'format': 'json'}
[i] requesting "https://en.wikipedia.org/w/api.php?action=query&prop=revisions&titles=Parallel computing&rvprop=timestamp|user|userid&rvlimit=max" with param {'rvcontinue': '20091122170409|327311047', 'format': 'json'}
[i] requesting "https://en.wikipedia.org/w/api.php?action=query&prop=revisions&titles=Parallel computing&rvprop=timestamp|user|userid&rvlimit=max" with param {'rvcontinue': '20071206015016|176063487', 'format': 'json'}
[i] requesting "https://en.wikipedia.org/w/api.php?action=query&prop=revisions&titles=Parallel computing&rvprop=timestamp|user|userid&rvlimit=max" with param {'rvcontinue': '20021108120211|411890', 'format': 'json'}


In [23]:
from dateutil.parser import parse as dtparse
from helper import is_bot_user, calc_freq

# Number of edits per editors
pass

# Frequency of edits (time between edits)
overall_edit_timestamp = []
overall_edit_nonbot_timestamp = []
for editor in dict_rev_info.values():
    is_bot = is_bot_user(editor[0]['user'], editor[0]['userid'])
    for rv in editor:
        ts = rv.get('timestamp')
        ts = dtparse(ts).timestamp()
        rv['timestamp_parsed'] = ts
        overall_edit_timestamp.append(ts)
        if not is_bot:
            overall_edit_nonbot_timestamp.append(ts)

freq_overall = calc_freq(overall_edit_timestamp)
freq_overall_nonbot = calc_freq(overall_edit_nonbot_timestamp)

freq_per_editor = {}
# edit freq per editor
for uid, revisions in dict_rev_info.items():
    freq_per_editor[uid] = calc_freq([rv['timestamp_parsed'] for rv in revisions])

## Talk-Page Posts

In [24]:
from WikiComponents import WikiBrowser

html_root = WikiBrowser.get_talk_page(article.id_)
len(html_root.find('div', id='bodyContent').find_all('h2'))

7

## Number of editors posting on talk-pages

In [25]:
# Use an API provided by XTools Wiki-Project
import requests

_url = 'https://xtools.wmflabs.org/api/page/articleinfo/en.wikipedia.org/Talk:{article_id}'
resp = requests.get(_url.format(article_id=article.id_)).json()
resp['editors']

74